# Preparation

In [101]:
import pandas as pd
from scipy.stats import entropy
import numpy as np
import collections
import pandas as pd
import time
import random
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo   
import math
import numpy as np
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import mutual_info_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import accuracy_score
from scipy.optimize import linear_sum_assignment
import numpy as np


def order(attri,i,datax):
    for j in range(len(datax[i])):
        if(attri == datax[i][j]):
            return j
        
def attri(x, i, datax):
    return datax[i][x]
def get_s(X, loca):
    s = []
    if(loca > X.shape[1]):
        return s
    else:
        for i in range(X.shape[1]):
            if(i>=loca):
                s.append(X.iloc[:, i].value_counts().index)
        for i in range(X.shape[1]-loca):
            s[i] = np.sort(s[i]) 
        return s 

def caa(y_true, y_pred):
    """
    accuracy
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    contingency = np.zeros((np.max(y_true) + 1, np.max(y_pred) + 1))
    for i in range(len(y_true)):
        contingency[y_true[i], y_pred[i]] += 1
    
    row_ind, col_ind = linear_sum_assignment(-contingency)
    
    mapping = {col: row for col, row in zip(col_ind, row_ind)}
    y_pred_mapped = np.array([mapping[cluster] for cluster in y_pred])

    return accuracy_score(y_true, y_pred_mapped)
    
def nmi(y,cluster_labels,real): 
    y = y.squeeze() 
    for i in range(len(real)):
        for j in range(len(real)):
            if(cluster_labels[i] == j):
                cluster_labels[i] = real[j]

    mi = mutual_info_score(y, cluster_labels)

    h_y = entropy(np.unique(y, return_counts=True)[1] / len(y))
    h_y_pred = entropy(np.unique(cluster_labels, return_counts=True)[1] / len(cluster_labels))

    nmi = mi / max(h_y, h_y_pred)

    print(f"NMI: {nmi:.3f}")
    return nmi


def nm_vo(y, cluster_labels):
    y = y.squeeze() 
    vi1s = mutual_info_score(y, cluster_labels)
    h_y = entropy(np.unique(y, return_counts=True)[1] / len(y))
    h_y_pred = entropy(np.unique(cluster_labels, return_counts=True)[1] / len(cluster_labels))
    return vi1s / max(h_y, h_y_pred) 

def ARI(cluster_labels,x):
    x = x.squeeze()
    ari = adjusted_rand_score(cluster_labels, x)
    print(f"Adjusted Rand Index (ARI): {ari:.3f}")
    return ari

# CADM

In [102]:
def weighted_mutual_infoa(cen, q, attr_idx, intra_dis, opt):
    gama_cen = intra_dis[attr_idx][cen]
    if opt:
        gama_q = intra_dis[attr_idx][q]
    else:
        gama_q = 1/intra_dis[attr_idx][q]
        
    mi_pq = gama_cen + gama_q
    
    return mi_pq

def pairwise_distance_ca2(X, Cen, k1, data, location, datax, importance_freq, intra_dis, opt, attribute_ranks=None):
    """
    CVD
    :param X: sample
    :param Cen: centroid
    :return: the distance between two objects
    """
    n = X.shape[1]
    distance = 0
    
    p_array = X[0]
    qc_array = Cen[0]
    
    for i in range(n):
        p = p_array[i]
        qc = qc_array[i]
        
        if i >= location:
            if p == qc:
                continue  
            else:
                
                if attribute_ranks and i in attribute_ranks and p in attribute_ranks[i] and qc in attribute_ranks[i]:
                    rank1 = attribute_ranks[i][p]
                    rankc = attribute_ranks[i][qc]
                else:
                    rank1 = order(p, i-location, datax)
                    rankc = order(qc, i-location, datax)
                
                diff = abs(rank1-rankc)
                first = min(rank1, rankc)
                gama_cen = intra_dis[i][qc]
                
                mutual_info_sum = 0
                for j in range(0, diff+1):
                    loc1 = first + j
                    it = attri(loc1, i-location, datax)
                    
                    if it != qc:
                        mutual_info = weighted_mutual_infoa(qc, it, i, intra_dis, opt) # CVD
                        mutual_info_sum += mutual_info
                
                distance += mutual_info_sum
                distance += importance_freq[i]*importance_freq[i] #CAI
        else:
            if p == qc:
                continue 
            else:
                mutual_info = weighted_mutual_infoa(qc, p, i, intra_dis, opt)
                distance += mutual_info + importance_freq[i]*importance_freq[i]
            
    return distance

def cluster_data_ca2(rs, df, location, datax, k1):   

    df_numpy = df.values
    
    attribute_ranks = {}
    for i in range(location, df.shape[1]):
        idx = i - location
        if idx < len(datax):
            attribute_ranks[i] = {datax[idx][j]: j for j in range(len(datax[idx]))}
    
    k = k1

    random_indices = np.random.RandomState(rs).choice(df.shape[0], k, replace=False)
    centroids = df_numpy[random_indices]
    
    importance_freq = []
    intra_dis = []
    
    for i1 in range(k):
        importance_freq.append({})
        intra_dis.append([])
        for j in range(df.shape[1]):
            intra_dis[i1].append({})
                       
    labels = np.zeros(df.shape[0], dtype=int)
    cns = df.shape[0]
    
    min_distance = np.full(df.shape[0], np.inf)
    
    columns = df.columns
    
    for turn in range(100):
        if turn > 10: 
            prev_labels = labels.copy()
        
        stable = True
        opt = (turn == 0)
        
        for j in range(k):
            if turn > 0:
                cluster_mask = (labels == j)
                cluster_points_df = df[cluster_mask]
            else:
                cluster_points_df = df
            
            for i in range(cluster_points_df.shape[1]):
                value_counts = cluster_points_df.iloc[:, i].value_counts()
                for val, count in value_counts.items():
                    intra_dis[j][i][val] = count/cns
            
            s_t = 0
            column_modes = []
            for i1 in range(cluster_points_df.shape[1]):
                counter = collections.Counter(cluster_points_df.iloc[:, i1])
                most_common = counter.most_common(1)[0]
                column_modes.append(most_common)
                s_t += most_common[1]
            
            for i1 in range(cluster_points_df.shape[1]):
                importance_freq[j][i1] = column_modes[i1][1]/s_t
        
        if df.shape[0] > 1000:
            batch_size = min(100, df.shape[0] // 10)  
            for batch_start in range(0, df.shape[0], batch_size):
                batch_end = min(batch_start + batch_size, df.shape[0])
                batch_indices = list(range(batch_start, batch_end))
                
                distances = []
                for i in batch_indices:
                    for j in range(k):
                        dist = pairwise_distance_ca2(
                            df_numpy[i:i+1], 
                            centroids[j:j+1], 
                            k1, df, location, 
                            datax, 
                            importance_freq[j], 
                            intra_dis[j], 
                            opt,
                            attribute_ranks
                        )
                        distances.append(dist)
                
                distances = np.array(distances).reshape(len(batch_indices), k)
                
                new_labels = np.argmin(distances, axis=1)
                new_min_distances = np.min(distances, axis=1)
                
                for idx, i in enumerate(batch_indices):
                    if new_min_distances[idx] < min_distance[i]:
                        min_distance[i] = new_min_distances[idx]
                        if labels[i] != new_labels[idx]:
                            labels[i] = new_labels[idx]
                            stable = False
        else:
            for i in range(df.shape[0]):
                for j in range(k):
                    distance = pairwise_distance_ca2(
                        df_numpy[i:i+1], 
                        centroids[j:j+1], 
                        k1, df, location, 
                        datax, 
                        importance_freq[j], 
                        intra_dis[j], 
                        opt,
                        attribute_ranks
                    )
                    if distance < min_distance[i]:
                        min_distance[i] = distance
                        if labels[i] != j:
                            labels[i] = j
                            stable = False
        
        for j in range(k):
            cluster_mask = (labels == j)
            if np.any(cluster_mask):
                cluster_points = df[cluster_mask]
                new_centroids = []
                
                for col in columns:
                    counter = collections.Counter(cluster_points[col])
                    mode = counter.most_common(1)[0][0]
                    new_centroids.append(mode)
                
                centroids[j] = np.array(new_centroids)
        
        if turn > 10:
            change_ratio = np.sum(labels != prev_labels) / df.shape[0]
            if change_ratio < 0.0001: 
                break
                
        if stable:
            break

    return labels, centroids


# CADM-m

In [103]:
def pairwise_distance_cah2(X, Cen, k1, data, intloc, location, datax, importance_freq, intra_dis, opt, attribute_ranks=None):
    #CVD
    
    n = X.shape[1]
    distance = 0
    
    p_array = X[0]
    qc_array = Cen[0]
    
    for i in range(n):
        p = p_array[i]
        qc = qc_array[i]
        
        if i < intloc:
            if p == qc:
                continue 
            else:
                distance = abs(int(p)-int(qc)) + 1
                
        
        if i >= location:
            if p == qc:
                continue 
            else:
                if attribute_ranks and i in attribute_ranks and p in attribute_ranks[i] and qc in attribute_ranks[i]:
                    rank1 = attribute_ranks[i][p]
                    rankc = attribute_ranks[i][qc]
                else:
                    rank1 = order(p, i-location, datax)
                    rankc = order(qc, i-location, datax)
                
                diff = abs(rank1-rankc)
                first = min(rank1, rankc)
                gama_cen = intra_dis[i][qc]
                
                mutual_info_sum = 0
                for j in range(0, diff+1):
                    loc1 = first + j
                    it = attri(loc1, i-location, datax)
                    
                    if it != qc:
                        mutual_info = weighted_mutual_infoa(qc, it, i, intra_dis, opt)
                        mutual_info_sum += mutual_info
                
                distance += mutual_info_sum
                distance += importance_freq[i]*importance_freq[i]
                
        else:
            if p == qc:
                continue  
            
    return distance

def cluster_data_cah2(rs, df, intloc, location, datax, k1):   
    
    df_numpy = df.values

    attribute_ranks = {}
    for i in range(location, df.shape[1]):
        idx = i - location
        if idx < len(datax):
            attribute_ranks[i] = {datax[idx][j]: j for j in range(len(datax[idx]))}
    
    k = k1
    random_indices = np.random.RandomState(rs).choice(df.shape[0], k, replace=False)
    centroids = df_numpy[random_indices]
    
    importance_freq = []
    intra_dis = []
    
    for i1 in range(k):
        importance_freq.append({})
        intra_dis.append([])
        for j in range(df.shape[1]):
            intra_dis[i1].append({})
                       
    labels = np.zeros(df.shape[0], dtype=int)
    cns = df.shape[0]
    
    min_distance = np.full(df.shape[0], np.inf)
    
    columns = df.columns
    
    for turn in range(100):
        if turn > 10: 
            prev_labels = labels.copy()
        
        stable = True
        opt = (turn == 0)

        for j in range(k):
            if turn > 0:
                cluster_mask = (labels == j)
                cluster_points_df = df[cluster_mask]
            else:
                cluster_points_df = df
            
            for i in range(cluster_points_df.shape[1]):
                value_counts = cluster_points_df.iloc[:, i].value_counts()
                for val, count in value_counts.items():
                    intra_dis[j][i][val] = count/cns
                    
            s_t = 0
            column_modes = []
            for i1 in range(cluster_points_df.shape[1]):
                counter = collections.Counter(cluster_points_df.iloc[:, i1])
                most_common = counter.most_common(1)[0]
                column_modes.append(most_common)
                s_t += most_common[1]
            
            for i1 in range(cluster_points_df.shape[1]):
                importance_freq[j][i1] = column_modes[i1][1]/s_t

        if df.shape[0] > 1000:
            batch_size = min(100, df.shape[0] // 10)  
            for batch_start in range(0, df.shape[0], batch_size):
                batch_end = min(batch_start + batch_size, df.shape[0])
                batch_indices = list(range(batch_start, batch_end))
                distances = []
                for i in batch_indices:
                    for j in range(k):
                        dist = pairwise_distance_cah2(
                            df_numpy[i:i+1], 
                            centroids[j:j+1], 
                            k1, df,intloc, location, 
                            datax, 
                            importance_freq[j], 
                            intra_dis[j], 
                            opt,
                            attribute_ranks
                        )
                        distances.append(dist)
                
                distances = np.array(distances).reshape(len(batch_indices), k)
                
                new_labels = np.argmin(distances, axis=1)
                new_min_distances = np.min(distances, axis=1)
            
                for idx, i in enumerate(batch_indices):
                    if new_min_distances[idx] < min_distance[i]:
                        min_distance[i] = new_min_distances[idx]
                        if labels[i] != new_labels[idx]:
                            labels[i] = new_labels[idx]
                            stable = False
        else:
            for i in range(df.shape[0]):
                for j in range(k):
                    distance = pairwise_distance_cah2(
                        df_numpy[i:i+1], 
                        centroids[j:j+1], 
                        k1, df, intloc, location, 
                        datax, 
                        importance_freq[j], 
                        intra_dis[j], 
                        opt,
                        attribute_ranks
                    )
                    if distance < min_distance[i]:
                        min_distance[i] = distance
                        if labels[i] != j:
                            labels[i] = j
                            stable = False
        
        for j in range(k):
            cluster_mask = (labels == j)
            if np.any(cluster_mask):
                cluster_points = df[cluster_mask]
                new_centroids = []
            
                for col in columns:
                    counter = collections.Counter(cluster_points[col])
                    mode = counter.most_common(1)[0][0]
                    new_centroids.append(mode)
                
                centroids[j] = np.array(new_centroids)

        if turn > 10:
            change_ratio = np.sum(labels != prev_labels) / df.shape[0]
            if change_ratio < 0.01: 
                break
                
        if stable:
            break

    return labels, centroids

# HA

In [104]:
# fetch dataset 

hayes_roth = fetch_ucirepo(id=44) 
  
# data (as pandas dataframes) 

X = hayes_roth.data.features 
y = hayes_roth.data.targets 
# delete null
X = X.loc[~y.isnull().any(axis=1)]
y = y.loc[~y.isnull().any(axis=1)]

In [105]:
od = 2
for i in range(1):
    cluster_labels3, centroids = cluster_data_ca2(44,X,od,get_s(X, od),3)
    haca3, real = cal(X,y,cluster_labels3)
    haari3 =ARI(cluster_labels3,y)
    hanmi3 = nmi(y,cluster_labels3,real)
print(f"Clustering Accuracy: {haca3:.3f}")
# print(f"Adjusted Rand Index (ARI): {haari3:.3f}")
# print(f"(NMI): {hanmi3:.3f}")

Adjusted Rand Index (ARI): 0.085
NMI: 0.098
Clustering Accuracy: 0.530


# LS

In [106]:
# fetch dataset 
lenses = fetch_ucirepo(id=58) 
  
# # data (as pandas dataframes) 
X = lenses.data.features 
y = lenses.data.targets 

In [107]:
od = 4
for i in range(1):
    cluster_labels3, centroids = cluster_data_ca2(999,X,od,get_s(X, od),3)
    lsca3, real = cal(X,y,cluster_labels3)
    lsari3 =ARI(cluster_labels3,y)
    lsnmi3 = nmi(y,cluster_labels3,real)

print(f"Clustering Accuracy: {lsca3:.3f}")

Adjusted Rand Index (ARI): 0.481
NMI: 0.327
Clustering Accuracy: 0.750


# VT

In [108]:
# fetch dataset 
congressional_voting_records = fetch_ucirepo(id=105) 
  
# data (as pandas dataframes) 
X = congressional_voting_records.data.features 
y = congressional_voting_records.data.targets 
X = X.dropna()
y = y.loc[X.index]
y = y.replace({'democrat': 0, 'republican': 1})

In [109]:
od = 17
for i in range(1):
    cluster_labels3, centroids = cluster_data_ca2(72,X,od,get_s(X, od),2)
    vtca3, real = cal(X,y,cluster_labels3)
    vtari3 =ARI(cluster_labels3,y)
    vtnmi3 = nmi(y,cluster_labels3,real)

print(f"Clustering Accuracy: {vtca3:.3f}")

Adjusted Rand Index (ARI): 0.655
NMI: 0.573
Clustering Accuracy: 0.905


# SM

In [110]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
secondary_mushroom = fetch_ucirepo(id=848) 
  
# data (as pandas dataframes) 
X = secondary_mushroom.data.features 
y = secondary_mushroom.data.targets 
  
X_cleaned = X.dropna(axis=1)
X_cleaned = X_cleaned.drop(X_cleaned.columns[[0, 5, 6]], axis=1)
y_replaced = y.replace({'p': 1, 'e': 2})

In [111]:
od = 0
for i in range(1):
    start_time = time.time()
    cluster_labels3, centroids3 = cluster_data_ca2(999,X_cleaned,od,get_s(X_cleaned, od),2)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f'Execution Time: {execution_time:.4f} seconds')
    y1 = y_replaced
    y2 = y_replaced.squeeze()  #
    y3 = y_replaced.squeeze()  #
    smca3 = caa(y1, cluster_labels3)
    smari3 =adjusted_rand_score(y2,cluster_labels3)
    smnmi3 =  normalized_mutual_info_score(y3, cluster_labels3)
print(f"Clustering Accuracy: {smca3:.3f}")
print(f"Adjusted Rand Index (ARI): {smari3:.3f}")
print(f"(NMI): {smnmi3:.3f}")    

Execution Time: 10.3323 seconds
Clustering Accuracy: 0.553
Adjusted Rand Index (ARI): 0.011
(NMI): 0.007


# NS

In [112]:
nursery = fetch_ucirepo(id=76) 
  
# data (as pandas dataframes) 
X = nursery.data.features 
y = nursery.data.targets 
datax = [['usual', 'pretentious', 'great_pret'],
 ['proper', 'less_proper', 'improper', 'critical', 'very_crit'],
 ['complete', 'completed', 'incomplete', 'foster'],
 ['1', '2', '3', 'more'],
 ['convenient', 'less_conv', 'critical'],
 ['inconv', 'convenient'],
 ['nonprob', 'slightly_prob', 'problematic'],
 ['not_recom', 'recommended', 'priority']] # order information

In [113]:
od = 0
start_time = time.time()
cluster_labels3, centroids3 = cluster_data_ca2(999, X, od, datax, 4)

end_time = time.time()

execution_time = end_time - start_time
print(f'Execution Time: {execution_time:.4f} seconds')

y2 = y.squeeze()  #
y3 = y.squeeze()  #
nsca3, real = cal(X, y, cluster_labels3)
print(f"Clustering Accuracy: {nsca3:.3f}")
nsari3 = adjusted_rand_score(y2, cluster_labels3)
print(f"Adjusted Rand Index (ARI): {nsari3:.3f}")
nsnmi3 = normalized_mutual_info_score(y3,cluster_labels3)
print(f"(NMI): {nsnmi3:.3f}")


Execution Time: 3.8123 seconds
Clustering Accuracy: 0.441
Adjusted Rand Index (ARI): 0.108
(NMI): 0.103


# HF

In [114]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
heart_failure_clinical_records = fetch_ucirepo(id=519) 
  
# data (as pandas dataframes) 
X = heart_failure_clinical_records.data.features 
y = heart_failure_clinical_records.data.targets 
columns_to_move = [2, 4, 6,7, 8, 11]  

remaining_columns = [i for i in range(X.shape[1]) if i not in columns_to_move]

new_column_order = [X.columns[i] for i in columns_to_move] + [X.columns[i] for i in remaining_columns]

X_reordered = X[new_column_order]

In [115]:
intloc = 6
od = 6
for i in range(1):
    cluster_labels, centroids = cluster_data_cah2(99, X_reordered, 6, 6,get_s(X_reordered, od),2) #44
    y2 = y.squeeze()  #
    y3 = y.squeeze()  #
    
    ca = caa(y, cluster_labels)
    print(f"Clustering Accuracy: {ca:.3f}")
    ari = adjusted_rand_score(y2, cluster_labels)
    print(f"Adjusted Rand Index (ARI): {ari:.3f}")
    hi1 = normalized_mutual_info_score(y3,cluster_labels)
    print(f"(NMI): {hi1:.3f}")


Clustering Accuracy: 0.742
Adjusted Rand Index (ARI): 0.157
(NMI): 0.145
